# Python Algorithms
## Chapter 9 From A to B with Edsger and Friends
### Propagating Knowledge

In [33]:
a, b, c, d, e, f, g, h = list(range(8))
G = {
 a: {b:2, c:1, d:3, e:9, f:4},
 b: {c:4, e:3},
 c: {d:8},
 d: {e:7},
 e: {f:5},
 f: {c:2, g:2, h:2},
 g: {f:1, h:6},
 h: {f:9, g:8}
}

In [5]:
inf = float('inf')
def relax(Graph, start, end, estimate:dict, parent):
    update = estimate.get(start,inf) + Graph[start][end] # a possible better path, current estimate to start and a path to end
    if update < estimate.get(end,inf): # really better?
        estimate[end], parent[end] = update, start
        return True 

### Relaxing like Crazy
After k rounds of relaxing every edge in the graph, we know that all shortest paths of consisting of k edges have been completed. Following our earlier reasoning, for a graph with n nodes and m edges, 
it will require at most n-1 rounds until we’re done, giving us a running time of $\Theta(nm)$. Of course, this need only be the worst-case running time, if we add a check: Has anything changed in the last round? If nothing changed, there’s no point in continuing. We might even be tempted to drop the whole n-1 count and only rely on this check. After all, we’ve just reasoned that we’ll never need more than n-1 rounds, so the check will eventually halt the algorithm.   
If we have no negative cycles, the “no change” condition will work just fine, but throw in a negative cycle, and our estimates can keep improving forever. So ... as long as we allow negative edges (and why wouldn’t we?), we need the iteration count as a safeguard. 

In [11]:
def BellmanFord(G,s):
    estimate,parent = {s:0},{}
    for round in G:
        change = False
        for start in G:  #len(G) rounds
            for end in G[start]:
                if relax(G,start,end,estimate,parent):
                    change = True
        if not change:
                break
    else:
        raise ValueError('negative cycle')
    return estimate,parent
BellmanFord(G,a)

({0: 0, 1: 2, 2: 1, 3: 3, 4: 5, 5: 4, 6: 6, 7: 6},
 {1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 5, 7: 5})

### Finding the Hidden DAG

In [20]:
from heapq import heappush, heappop
def dijkstra(G,s):
    estimate, tree, queue, visited = {s:0}, {}, [(0,s)],set()
    while queue:
        _,u = heappop(queue)
        if u in visited:
            continue
        visited.add(u)
        for v in G[u]:
            relax(G,u,v,estimate,tree)
            heappush(queue,(estimate[v],v))
    return estimate,tree
dijkstra(G,a)

({0: 0, 1: 2, 2: 1, 3: 3, 4: 5, 5: 4, 6: 6, 7: 6},
 {1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 5, 7: 5})

### All Against All
When solving the all-pairs shortest paths problem for sparse graphs, simply using Dijkstra’s algorithm from every node is, in fact, a really good solution. That in itself doesn’t exactly motivate a new algorithm ... but the trouble is that Dijkstra’s algorithm doesn’t permit negative edges. For the single-source shortest path problem, there isn’t much we can do about that, except use Bellman-Ford instead. For the all-pairs problem, though, we can permit ourselves some initial preprocessing to make all the weights positive.  
The idea is to add a new node s, with zero-weight edges to all existing nodes, and then to run Bellman-Ford from 
s. This will give us a distance—let’s call it h(v)—from s to each node v in our graph. We can then use h to adjust the weight of every edge: We define the new weight as follows: w’(u,v) = w(u,v) + h(u) - h(v). This definition has two very useful properties. First, it guarantees us that every new weight w’(u,v) is nonnegative (this follows from the triangle inequality). Second, we’re not messing up our problem! That is, if we find the shortest paths with these new weights, those paths will also be shortest paths (with other lengths, though) with the original weights. 

In [21]:
from copy import deepcopy
def johnson(G):
    G = deepcopy(G)
    s = object()
    G[s] = {v:0 for v in G}
    h,_ = BellmanFord(G,s)
    del G[s]
    for start in G:
        for end in G[start]:
            G[start][end] += h[start] - h[end]
    estimate,parent = {},{}
    for start in G:
        estimate[start],parent[start] = dijkstra(G,start)
        for end in G[start]:
            estimate[start][end] += h[end] - h[start]
    return estimate,parent
johnson(G)

({0: {0: 0, 1: 2, 2: 1, 3: 3, 4: 5, 5: 4, 6: 6, 7: 6},
  1: {1: 0, 2: 4, 4: 3, 5: 8, 3: 12, 6: 10, 7: 10},
  2: {2: 0, 3: 8, 4: 15, 5: 20, 6: 22, 7: 22},
  3: {3: 0, 4: 7, 5: 12, 2: 14, 6: 14, 7: 14},
  4: {4: 0, 5: 5, 2: 7, 6: 7, 7: 7, 3: 15},
  5: {5: 0, 2: 2, 6: 2, 7: 2, 3: 10, 4: 17},
  6: {6: 0, 5: 1, 7: 3, 2: 3, 3: 11, 4: 18},
  7: {7: 0, 5: 9, 6: 8, 2: 11, 3: 19, 4: 26}},
 {0: {1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 5, 7: 5},
  1: {2: 1, 4: 1, 5: 4, 3: 2, 6: 5, 7: 5},
  2: {3: 2, 4: 3, 5: 4, 6: 5, 7: 5},
  3: {4: 3, 5: 4, 2: 5, 6: 5, 7: 5},
  4: {5: 4, 2: 5, 6: 5, 7: 5, 3: 2},
  5: {2: 5, 6: 5, 7: 5, 3: 2, 4: 3},
  6: {5: 6, 7: 5, 2: 5, 3: 2, 4: 3},
  7: {5: 7, 6: 7, 2: 5, 3: 2, 4: 3}})


### Far-Fetched Subproblems
We arbitrarily order the nodes and restrict how many—that is, the k first—we’re allowed to use as intermediate nodes in forming our paths. We have now parametrized our subproblems using three parameters:
+ The starting node
+ The ending node
+ The highest node number we’re allowed to pass through  

Let $d(u, v,k)$ be the length of the shortest path that exists from node u to node v if you’re only allowed to use the $k$ first nodes as intermediate nodes. We can decompose the problem as follows:
$$
d(u, v, k) = \min(d(u, v,k−1), d(u,k,k−1) + d(k, v,k−1))
$$

In [27]:
from functools import wraps
def memo(func):
    cache = {}
    @wraps(func)
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

In [29]:
def rec_floyd(G):
    @memo
    def d(u,v,k):
        if k == 0:
            return G[u][v]
        return min(d(u,v,k-1),d(u,k,k-1)+d(k,v,k-1))
    return {(u,v):d(u,v,len(G)) for u in G for v in G}

In [36]:
def itr_floyd(G):
    D = deepcopy(G)
    for k in G:
        for u in G:
            for v in G:
                D[u][v] = min(D[u][v],D[u][k] + D[k][v])
    return D


### Meeting in the Middle
### Knowing Where You’re Going
In A*, we want to modify the edges in a similar fashion, but this time the goal isn’t to make the 
edges positive—we’re assuming they already are (as we’re building on Dijkstra’s algorithm). No, what we want is to guide the traversal in the right direction, by using information of where we’re going: We want to make edges moving away from our target node more expensive than those that take us closer to it.

In [ ]:
from heapq import heappop,heappush
def a_star(G,s,t,h):
    P,Q = {},[(h(s),None,s)]
    while Q:
        d,p,u = heappop(Q)
        if u in P:
            continue
        P[u] = p
        if u == t:
            return d-h(t),P
        for v in G[u]:
            w = G[u][v] - h(u) + h(v)
            heappush(Q,(d+w,u,v))
    return inf,None

Exercises
1. In some cases, discrepancies in exchange rates between currencies make it possible to exchange from one currency to another, continuing until one gets back to the original, having made a profit. How would you use the Bellman-Ford algorithm to detect the presence of such a situation?    
   You have to somehow modify the algorithm or the graph so the detection mechanism for negative additive 
cycles can be used to find multiplicative cycles where the product of the exchange rates ends up above 1. The easiest solution is to simply take transform all the weights by taking their logarithms and negating them. 
2. What happens in Dijkstra’s algorithm if more than one node has the same distance from the start 
node? Is it still correct?   
   It does not matter
3. Why is it a really bad idea to represent edge length using dummy nodes, like in Figure 9-3?  
   pseudopolynomial running time
4. What would the running time of Dijkstra’s algorithm be if you implemented it with an unsorted list 
instead of a binary heap?  
   It could be done in constant time
5. Why can we be certain that the adjusted weights in Johnson’s algorithm are nonnegative? Are 
there cases where things can go wrong?   
   If there are negative cycles, the bellman ford will detect it. 
6. In Johnson’s algorithm, the h function is based on the Bellman-Ford algorithm. Why can’t we just use an arbitrary function here? It would disappear in the telescoping sum anyway?   
   We can't guarantee nonnegative weights. 
7. Implement the memoized version of Floyd-Warshall so it saves memory in the same way as the iterative one.
8. Extend the memoized version of Floyd-Warshall to compute a P table, just like the iterative one.
9. How would you modify the Floyd-Warshall algorithm so it detects the presence of paths, rather than finding the shortest paths (Warshall’s algorithm)?
10. Why does correctness for the tighter stopping criterion for the bidirectional version of Dijkstra’s 
algorithm imply correctness for the original?
11. In the correctness proof for the bidirectional version of Dijkstra’s algorithm, I posited a hypothetical path that would be shorter than the best one we’d found so far and stated that it had to contain an edge (u,v) such that d(s,u) < l and d(v,t) < r. Why is this the case?
12. Rewrite bidir_dijkstra so it doesn’t require the input graph to be symmetric, with zero-weight 
self-edges.
13. Implement a bidirectional version of BFS.
14. Why is h(v) a lower bound on d(v,t) when w’ is feasible?